In [ ]:
!pip -U install transformers

In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis")
pipe("it is terrible")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model():
    model_id = "HuggingFaceH4/zephyr-7b-beta"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)
    return tokenizer, model

In [ ]:
from model.load_model import load_model
import torch

tokenizer, model = load_model()

def get_response(user_input):
    input_ids = tokenizer.encode(user_input, return_tensors="pt").cuda()
    output = model.generate(input_ids, max_new_tokens=150, do_sample=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [ ]:
import csv
import os

def store_feedback(question, answer, feedback):
    if feedback.lower() == "negative":
        with open("feedback/feedback_data.csv", "a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([question, answer])

In [ ]:
import pandas as pd
from datasets import Dataset

def get_dataset():
    df = pd.read_csv("feedback/feedback_data.csv", names=["question", "answer"])
    df["text"] = df.apply(lambda row: f"User: {row['question']} \nAI: {row['answer']}", axis=1)
    dataset = Dataset.from_pandas(df)
    return dataset.train_test_split(test_size=0.1)

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from model.load_model import load_model
from training.prepare_dataset import get_dataset

def fine_tune_on_feedback():
    tokenizer, model = load_model()
    dataset = get_dataset()

    peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
    model = get_peft_model(model, peft_config)

    args = TrainingArguments(
        output_dir="./checkpoints",
        evaluation_strategy="epoch",
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        num_train_epochs=2,
        save_strategy="epoch",
        logging_steps=10,
        fp16=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    )

    trainer.train()

In [ ]:
from inference.chat import get_response
from feedback.store_feedback import store_feedback

while True:
    question = input("You: ")
    if question.lower() == "exit":
        break
    answer = get_response(question)
    print("AI:", answer)
    feedback = input("Was the answer helpful? (positive/negative): ")
    store_feedback(question, answer, feedback)